<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Доказательство" data-toc-modified-id="Доказательство-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Доказательство</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

## Загрузка данных

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

In [6]:
df = pd.read_csv('/datasets/insurance.csv')
df

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [8]:
features = df.drop('Страховые выплаты',axis=1)
target = df['Страховые выплаты']

In [9]:
matrix_features = features.values
matrix_features

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [10]:
vector_target = target.values
vector_target

array([0, 1, 0, ..., 0, 0, 0])

Предоставленные данные не дублируются, в них отсутствуют пропуски и выбросы. Данные представлены в нужном нам численном формате, который подходит для решения задачи регрессии. Все признаки числовые.

Можно приступать к обучению модели.

## Умножение матриц

In [11]:
model = LinearRegression()
model.fit(matrix_features,vector_target)
predictions = model.predict(matrix_features)
metrics = r2_score(vector_target,predictions)
metrics

0.4249455028666801

In [12]:
def rnd_matrix(n):
    random_matrix = np.random.normal(size=(n, n))
    try:
        np.linalg.inv(random_matrix) # выполняется операция создания обратной матрицы
    except LinAlgError: # если возникает ошибка, т.е. обратной матрицы не существует, выполняется еще одна попытка
        rnd_matrix(n)
    return random_matrix

In [13]:
random_matrix = rnd_matrix(4)
encoded_matrix = matrix_features @ random_matrix
encoded_matrix.shape

(5000, 4)

In [14]:
model = LinearRegression()
model.fit(encoded_matrix,vector_target)
predictions = model.predict(encoded_matrix)
metrics = r2_score(vector_target,predictions)
metrics

0.4249455028666801

Использовав рэндомную матрицу , нам удалось зашифровать данные и не потерять в качестве метрики(в данном случае R2) при обучении и предсказании модели.

Однако, что бы доказать что данные преобразования будут верны при умножении на любую другую матрицу, необходимо обратиться к математическим формулам и доказать что данные линейные преобразования не влияют на обучение/предсказания модели Линейной регресии. 

## Доказательство

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$





**Пусть предсказание на преобразованной выборке будет равно:**
$$
a_1= X_1w_1, где X_1=XP
$$


Задача обучения:

$$
w_1 = \arg\min_w MSE(X_1w_1, y)
$$

Формула обучения:

$$
w_1 = (X_1^T X_1)^{-1} X_1^T y
$$

Требуется доказать,  что $a_1=a$ 

Доказательство:

Сналача преобразуем показатель $w_1$:

$$
((XP)^T XP)^{-1} (XP)^T y = (P^T X^T XP)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^{-1}P)^T X^T y = P^{-1} (X^T X)^{-1}X^T y
$$

Далее, подставим в формулу $a = a_1$ получившееся выражение:
$$
Xw = X_1w_1
$$
$$
Xw = XPw_1
$$
$$
X (X^T X)^{-1} X^T y = X P P^{-1} (X^T X)^{-1}X^T y
$$
$$
X (X^T X)^{-1} X^T y = X (X^T X)^{-1}X^T y
$$

Таким образом, мы можем утверждать,что линейные преобразования с исходными данными не влияют на обучение и предсказание моделей линейной регрессии.